In [2]:
from Abbrev import ABBREVIATIONS
from contractions import CONTRACTION_MAP
from wordfreq import WordFrequency


import pandas.io.sql as pds
import pandas as pd
import numpy as np
import pyodbc
import scipy.sparse as sp
from numpy.linalg import norm
import nltk
import re
import os
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import gensim
from gensim.models import word2vec
from sklearn.manifold import TSNE
from pattern3.en import tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from __future__ import print_function
from sklearn.manifold import MDS
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
#import func
import importlib
#importlib.reload(func)

def connect(query):
    conn_str = (
    r'DRIVER={ODBC Driver 13 for SQL Server};'
    r'SERVER=ICWDSADASCI01VG;'
    r'DATABASE=CernerNotes;'
    r'Trusted_Connection=yes;')
    cnxn = pyodbc.connect(conn_str)
    #query ='"""{}"""'.format(query)
    query =query 
    df = pds.read_sql(query, cnxn)
    return df
    
def Dataframe(corpus):
    for x in corpus:
        dfNotes = corpus['NoteText']
        dfID=corpus['EVENT_ID']
    return dfID, dfNotes

def Dataframe(corpus):
    dfID = [x for x in corpus['EVENT_ID']]
    dfNotes = [x for x in corpus['NoteText']]
   
    return dfID, dfNotes

def Transform(EVID,note=False,cluster=False):
    Model = pd.DataFrame({'EventID': EVID})
    Model.insert(0, 'Order_ID', range(1, 1+ len(Model)))
    ModelID = Model['Order_ID']
    ID = ModelID.tolist()

    if note:

        vector = { 'Event': ID,'EventID':EVID,'NoteText': note }
        frame = pd.DataFrame(vector, index = [ID] , columns = ['Event','EventID','NoteText'])
        return vector,frame
    elif cluster:

        vector = { 'Event': ID,'cluster': cluster }
        frame = pd.DataFrame(vector, index = [cluster] , columns = ['Event','cluster'])
        return vector,frame,ID
    else:
        vector = { 'Event': ID,'EventID':EVID}
        frame = pd.DataFrame(vector, index = [ID] , columns = ['Event','EventID'])
        return vector,frame

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also']
wnl = WordNetLemmatizer()

def remove_generic_text(note):
    clean = re.sub(
    '(Tel)*:*\s\(*\d{3}\)*\s*-*\d{3}-*\s*\d{4}\s|(Fax)*:*\s\(*\d{3}\)*\s*-*\d{3}-*\s*\d{4}\s|Med\.\sRec\.\s#\:\s[0-9]{2,13}|(DOB)*:*\s*\d{1,2}\/\d{1,2}\/\d{2,4}\s*|MRN:\s\d{1,8}|FIN:\s\d{1,12}|Accession #:\s[a-zA-Z0-9]{2,5}-[a-zA-Z0-9]{2,5}|Patient: [A-Za-z]{1,10}, [A-Za-z]{1,10}|Encounter\s#\:\s[0-9]{2,15}|Age: \d{1,3}|Sex: [A-Za-z]{1,10}| Gender: [A-Za-z]{1,10}|Patient Name: [A-Za-z]{1,10}, [A-Za-z]{1,10}|Author: [A-Za-z]{2,25}, [A-Za-z]{2,25}'," ",note)
    return clean

def tokenize_text(text):
    tokens = nltk.word_tokenize(text) 
    tokens = [token for token in tokens]
    return tokens

def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def replace_abbreviation(text):
    xxx=[]
    for line in text:
        new_line = line
        for word in line.split():
            abb = ABBREVIATIONS.get(word)
            #for f_key, f_value in ABBREVIATIONS.items():
            if abb is not None:
                new_line = new_line.replace(word,abb)
    xxx.append(new_line)
    print(xxx)

def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


            
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z0-9]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def normalize_corpus(corpus,tokenize=False):
    
    normalized_corpus = []    
    for text in corpus:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = remove_generic_text(text)  
        #text = replace_abbreviation(text)
        text = text.lower()
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)
        else:
            normalized_corpus.append(text)

    return normalized_corpus   

# def build_feature_matrix(documents, feature_type='tfidf', ngram_range=(1, 1), min_df=0.0, max_df=1.0,use_idf=False):

#     feature_type = feature_type.lower().strip()  
    
#     if feature_type == 'binary':
#         vectorizer = CountVectorizer(binary=True, min_df=min_df,
#                                      max_df=max_df, ngram_range=ngram_range)
#     elif feature_type == 'frequency':
#         vectorizer = CountVectorizer(binary=False, min_df=min_df,
#                                      max_df=max_df, ngram_range=ngram_range)
#     elif feature_type == 'tfidf':
#         vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, 
#                                      ngram_range=ngram_range,use_idf=use_idf)
#     else:
#         raise Exception("Possible values: 'binary', 'frequency', 'tfidf'")

#     feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
#     return vectorizer, feature_matrix


def compute_cosine_similarity(query_TFIDF, corpus_TFIDF, top_n=3):
    similarity = cosine_similarity(query_TFIDF, corpus_TFIDF).flatten()
    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3))for index in top_docs]
    return top_docs_with_score

def Tokenize_norm(note):
    toks = []
    for x in note:
        token = tokenize_text(x)
    toks.append(token)
    return toks
    





import warnings
warnings.filterwarnings('ignore') 


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
def bow_extractor(documents,ngram_range, min_df,max_df, use_idf=False):
      
    vectorizer = CountVectorizer(ngram_range=ngram_range, min_df=min_df)
    features = vectorizer.fit_transform(documents)
    f_names = vectorizer.get_feature_names()        
    return vectorizer, features, f_names

def display_features(features, feature_names):
    df = pd.DataFrame(data=features, columns=feature_names)
    print(df)

In [9]:
bow_vectorizer,bow_features,features_names = bow_extractor(norm_corpusTrain,ngram_range =(0,1),min_df =0.2,max_df=0.8)
test_features = bow_vectorizer.transform(norm_corpusTest)
test_features = test_features.todense()
tf = bow_features.todense()
tf = np.array(tf, dtype='float64')

In [29]:
#len(tf)

In [10]:
fnames=bow_vectorizer.get_feature_names()

In [ ]:
#display_features(tf,features_names)

In [ ]:
# for word in features_names:
#     for line in note:
#         for word in line.split():
#             wf = WordFrequency.get(word)
#             if wf is not None:
#                 tfidf = tf * idf + wf
#             else: tfidf = tf * idf
             

In [11]:
# build the document frequency matrix
df = np.diff(sp.csc_matrix(bow_features, copy=True).indptr)
df = 1 + df # to smoothen idf later

In [12]:
# compute inverse document frequencies
total_docs = 1 + len(norm_corpusTrain)
idf = 1.0 + np.log(float(total_docs) / df)

In [13]:
feature = bow_features.todense()

In [ ]:
#d#isplay_features(feature, features_names)

In [14]:
 # compute idf diagonal matrix  
total_features = bow_features.shape[1]
idf_diag = sp.spdiags(idf, diags=0, m=total_features, n=total_features)
idf = idf_diag.todense()

In [15]:
#compute tfidf feature matrix
tfidf = tf * idf
#compute L2 norms 
norms = norm(tfidf, axis=1)
# compute normalized tfidf
norm_Train_tfidf = tfidf / norms[:, None]

In [21]:
print(norm_Train_tfidf)

[[0.         0.         0.05426927 ... 0.         0.0250985  0.02793408]
 [0.01745629 0.04407441 0.0623576  ... 0.         0.03460704 0.01283896]
 [0.19081654 0.         0.1363275  ... 0.         0.         0.04678134]
 ...
 [0.         0.15026577 0.         ... 0.04773245 0.05243913 0.0583636 ]
 [0.         0.         0.09013458 ... 0.         0.04168552 0.        ]
 [0.05614869 0.         0.04011503 ... 0.         0.03710487 0.        ]]


In [ ]:
# compute tfidf using idf matrix from train corpus
test_tfidf = test_features*idf
test_norms = norm(test_tfidf, axis=1)
Test_tfidf = test_tfidf / test_norms[:, None]


In [ ]:
Test_tfidf.shape

In [5]:
dfTrain = connect("""SELECT TOP (300) EVENT_ID, NoteText FROM [CernerNotes].[dbo].[DecompressedNote] where NoteText LIKE N'%Breast C_%' OR NoteText LIKE N'%Metastatic Breast Cancer%' OR NoteText LIKE N'%Triple-Negative%' OR NoteText LIKE N'%tnbc%' AND(len([NoteText]) - LEN(replace([NoteText], ' ', '')) +1)<200  AND (len([NoteText]) - LEN(replace([NoteText], ' ', '')) +1)>180""")
dfTest = connect("""SELECT TOP (50) EVENT_ID, NoteText FROM [CernerNotes].[dbo].[DecompressedNote]  WHERE (len([NoteText]) - LEN(replace([NoteText], ' ', '')) +1)<200  AND (len([NoteText]) - LEN(replace([NoteText], ' ', '')) +1)>180""")


In [6]:
dfIDtr, dfNotestr =Dataframe(dfTrain)
dfIDtest, dfNotest =Dataframe(dfTest)

In [7]:
vectTrain,frameTrain = Transform(dfIDtr,dfNotestr)
vectTest,frameTest = Transform(dfIDtest,dfNotest)

In [8]:
norm_corpusTrain = normalize_corpus(dfNotestr,tokenize=False)
norm_corpusTest = normalize_corpus(dfNotest,tokenize=False)

In [ ]:
from nltk import FreqDist
train_common = unique_tokens(norm_corpusTrain)
test_common = unique_tokens(norm_corpusTest)
tr = unique(train_common)
ts = unique(test_common)
joint = tr + ts

In [ ]:
len(tr)

In [ ]:
joint_u = unique(joint)

In [ ]:
import csv
csvfile = "C:/Users/oolakunle/Desktop/output.csv"

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in joint_u:
        writer.writerow([val])    

In [ ]:
# tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpusTrain,feature_type='tfidf',ngram_range=(1, 3), min_df=0.0, max_df=1.0)
# Test_tfidf = tfidf_vectorizer.transform(norm_corpusTest)


In [ ]:
# tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpusTrain,norm_corpusTest,ngram_range=(0, 1), min_df=1)
# Test_tfidf = tfidf_vectorizer.transform(norm_corpusTest)


In [ ]:
#get Cosine similarity results for our example documents
sim = []
disim = []
print('Document Similarity Analysis using Cosine Similarity')
print ('='*60)

for index, E in enumerate(frameTest['EventID']):
    doc_tfidf = Test_tfidf[index]
    top_similar_docs = compute_cosine_similarity(doc_tfidf,norm_Train_tfidf, top_n=1)
    print ('Document Number',index+1 ,'With EventID:', E ) 
    print ('Top', len(top_similar_docs), 'similar docs:')
    print( '-'*40)

    for doc_index, sim_score in top_similar_docs:
         for A, B in enumerate(frameTrain['EventID']):     
                if doc_index+1 == A:

                    if sim_score<=1.00 and sim_score>0.10:

                            print('Similar')
                            print('Doc num: {} Similarity Score: {}\nDoc num: {}'.format(doc_index+1,sim_score,B))
                            sim.append(B)

                    elif sim_score<=0.10:
                        #if doc_index+1 == A:
                            print('No Similarity')
                            print('Doc num: {} Similarity Score: {}\nDoc num: {}'.format(doc_index+1,sim_score,B))
                            disim.append(B)


In [ ]:
doc_tfidf.shape


In [ ]:
norm_Train_tfidf.shape

In [ ]:
sample_1 = connect("""SELECT EVENT_ID, NoteText FROM [CernerNotes].[dbo].[DecompressedNote] where EVENT_ID =14009026858""")

In [ ]:
sample_2  = connect("""SELECT EVENT_ID, NoteText FROM [CernerNotes].[dbo].[DecompressedNote] where EVENT_ID =2505100836""")

In [ ]:
df_samp1_EID,df_samp1_Notes = Dataframe(sample_1)
df_samp2_EID,df_samp2_Notes = Dataframe(sample_2)

In [ ]:
df_samp1_Notes

In [ ]:
df_samp2_Notes

In [ ]:
s1 = normalize_corpus(df_samp1_Notes)
s2 = normalize_corpus(df_samp2_Notes)

In [ ]:
def unique_tokens(normalized):
    toks = Tokenize_norm(normalized)
    tokens = [item for sublist in toks for item in sublist]
    return tokens

In [ ]:
def unique(tokens):
    fdist=FreqDist(tokens)
    most_common = fdist.most_common()
    unique = []
    for x,y in most_common:
        if y>=1:
            unique.append(x)
    return unique

In [ ]:
ss1 = Tokenize_norm(s1)
ss2 =Tokenize_norm(s2)

In [ ]:
tokens1 = [item for sublist in ss1 for item in sublist]
tokens2 = [item for sublist in ss2 for item in sublist]

In [ ]:
from nltk import FreqDist
fdist1=FreqDist(tokens1)
fdist2=FreqDist(tokens2)

In [ ]:
most_common1 = fdist1.max()
most_common2 = fdist2.max()

In [ ]:
a = fdist1.most_common()
b = fdist2.most_common()
#print(a, end= "")


In [ ]:
#print(b, end= "")

In [ ]:
arr1 = []
for x,y in a:
    if y>=1:
        arr1.append(x)

In [ ]:
arr2 = []
for x,y in b:
    if y>=1:
        arr2.append(x)

In [ ]:
common= [x for x in arr1 if x in arr2]

In [ ]:
print(common, end= "") 

In [ ]:
len(tokens1)

In [ ]:
len(tokens2)

In [ ]:
#common

In [ ]:
len(common)

In [ ]:
# z = round((len(tokens2)/len(tokens1))*100)
# z

In [ ]:
x = round((len(common)/len(tokens1))*100)
x

In [ ]:
y = round((len(common)/len(tokens2))*100)
y

In [ ]:
#Visualization with MDS
totalvocab_tokenized = []
for i in norm_corpusTest:
    allwords_tokenized = tokenize_text(i)
    totalvocab_tokenized.extend(allwords_tokenized)


In [ ]:
#Create a dataframe with EventID as the Index
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_tokenized)


In [ ]:
tfidf_vectorizer, feature_matrix =build_feature_matrix(norm_corpusTest, feature_type='tfidf', ngram_range=(1, 4), min_df=0.2, max_df=0.8, use_idf=True)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
#Distance Using Cosine Similarity
dist = 1 - cosine_similarity(feature_matrix)

In [ ]:
#Creating clusters using kmeans
num_clusters = 10

km = KMeans(n_clusters=num_clusters)

%time km.fit(feature_matrix)

clusters = km.labels_.tolist()


In [ ]:
vector,frame,ID = Transform(dfIDtest,False,clusters)

In [ ]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    
    print("Words common to cluster %d:   " % i, end='')
    for ind in order_centroids[i, :10]:
        v = vocab_frame.loc[terms[ind].split(' ')].values.tolist()
        #x=v.astype(np.float)
        #x = np.array(v, dtype=np.float32)
        ##mv
        print(' %s' %v[0][0], end=',')
    print()
    print()
    print("Notes in cluster %d:  " % i, end='')
    for Event in frame.loc[i]['Event'].values.tolist():
        print('%s,' % Event, end='')
    print()
    print()



In [ ]:
#Multidimensional scaling
MDS()

mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

In [ ]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e', 5: '#800080', 6: '#000080', 7: '#00FFFF', 8: '#FF0000', 9: '#FFFF00'}

#set up cluster names using a dict
cluster_names = {0: '1', 1: '2', 2: '3',3: '4', 4: '5',5: '6',6: '7',7: '8',8: '9',9: '10'}
                 


In [ ]:
%matplotlib inline

In [ ]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=ID)) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(20, 12)) # set size
ax.margins(0.12) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=8, label=cluster_names[name], color=cluster_colors[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='False',      # ticks along the bottom edge are off
        top='False',         # ticks along the top edge are off
        labelbottom='False')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='False',      # ticks along the bottom edge are off
        top='False',         # ticks along the top edge are off
        labelleft='False')
    
ax.legend(numpoints=1)  #show legend with only 1 point

#add label in x,y position with the label as the film title
for i in range(len(df)):
    ax.text(df.loc[i]['x'], df.loc[i]['y'], df.loc[i]['title'], size=12)  

    
    
plt.show()

In [ ]:
#dfNotest[77]

In [ ]:
#dfNotest[71]

In [ ]:
#Principal Component Analysis